In [1]:
import pandas as pd
import os
from tqdm import tqdm
from utils import avg, evidence_to_mask

def to_data_df(df, data_dir):
    data_df = []
    columns = ['text', 'classification', 'rationale', 'query']
    for i in tqdm(range(len(df))):
        df_row = df.loc[i]
        doc_ids = df_row['docids']
        query = df_row['query']
        evidence_list = df_row['evidences']
        if evidence_list:
            evidence_list = [x for xx in evidence_list for x in xx]
        classification = df_row['classification']
        
        text = ''
        for doc in doc_ids:
            file = f'{data_dir}/docs/{doc}'
            if os.path.isfile(file):
                f = open(file, 'r', encoding="utf-8") 
                for line in f.readlines():
                    text += line.rstrip() + ' '
            else:
                print("???")
                print(file)
                quit()
        
        tokens = text.split()
        rationale_mask = evidence_to_mask(tokens, evidence_list)
        
        # joining text and query with [SEP]
#         QA = f"{query} [SEP] {text}"
        QA = f"{text}</s> {query}"
        rationale_mask = [1]*(len(query.split())+1) + rationale_mask
    
        data_df.append([QA, classification, rationale_mask, query])
    data_df = pd.DataFrame(data_df, columns=columns)
    return data_df
    
    data_df_shuffled=data_df.sample(frac=1).reset_index(drop=True)
    return data_df_shuffled

In [2]:
dataset = "boolq"

In [3]:
data_dir = f'../data/{dataset}'
train = pd.read_json(f'{data_dir}/train.jsonl', lines=True)
test = pd.read_json(f'{data_dir}/test.jsonl', lines=True)
val = pd.read_json(f'{data_dir}/val.jsonl', lines=True)

In [4]:
train_data_df = to_data_df(train, data_dir)
train_data_df.to_csv(f"{dataset}/train.csv",index_label="id")
test_data_df = to_data_df(test, data_dir)
test_data_df.to_csv(f"{dataset}/test.csv",index_label="id")
val_data_df = to_data_df(val, data_dir)
val_data_df.to_csv(f"{dataset}/val.csv",index_label="id")

100%|█████████████████████████████████████████████████████████████████████████████| 1491/1491 [00:03<00:00, 404.92it/s]


# SANITY CHECK

In [60]:
def rationale_check(text,rationale):
    tokens = text.split()
    out = ""
    for i, b in enumerate(rationale):
        if b:
           out += tokens[i] + " "
    return out

In [38]:
data_df = test_data_df
import json

def reduce_by_alpha(text, rationale, fidelity_type="sufficiency"):
    reduced_text = ""
    # whitespace tokenization
    tokens = text.split()

    for idx in range(len(tokens)):
        try:
            if fidelity_type == "sufficiency" and rationale[idx] >= 0.5:
                reduced_text = reduced_text + tokens[idx] + " "
            elif fidelity_type == "comprehensiveness" and rationale[idx] < 0.5:
                reduced_text = reduced_text + tokens[idx] + " "
        except Exception as e:
            if fidelity_type == "comprehensiveness":
                reduced_text = reduced_text + tokens[idx] + " "

    # removed the last space from the text
    if len(reduced_text) > 0:
        reduced_text = reduced_text[:-1]

    return reduced_text

data_df = data_df[data_df['rationale'].notna()]
data_df.reset_index(drop=True, inplace=True)


data_df["sufficiency_text"] = data_df[
    ["text", "rationale"]].apply(lambda s: reduce_by_alpha(*s, fidelity_type="sufficiency"), axis=1)
data_df["comprehensiveness_text"] = data_df[
    ["text", "rationale"]].apply(lambda s: reduce_by_alpha(*s, fidelity_type="comprehensiveness"), axis=1)

In [61]:
rationale_check(train_data_df.iloc[2]['text'],train_data_df.iloc[2]['rationale'])

'is windows movie maker part of windows essentials [SEP] WINDOWS MOVIE MAKER Windows Movie Maker ( formerly known as Windows Live Movie Maker in Windows 7 ) was a video editing software by Microsoft . It was a part of Windows Essentials software suite and offered the ability to create and edit videos as well as to publish them on OneDrive , Facebook , Vimeo , YouTube , and Flickr '

In [71]:
train['classification'][0]

'True'

# DATA STATS

In [72]:
def generate_class_stats(train_df, test_df, val_df):
    text_lens_0 = []
    text_lens_1 = []
    rationale_lens_0 = []
    rationale_lens_1 = []
    rationale_percent_0 = []
    rationale_percent_1 = []
    class_distribution = [0,0]
    for df in [train_df, test_df, val_df]:
        for i in range(len(df)):
            df_row = df.loc[i]
            clas = df_row['classification']
            text = df_row['text']
            rationale = df_row['rationale']
            query = df_row['query']
            
            query_len = len(query.split())
            text_len = len(text.split()) - query_len - 1
            rationale_len = rationale.count(1) - query_len - 1
            rationale_percent = rationale_len/text_len
            if clas == "False":
                text_lens_0.append(text_len)
                rationale_lens_0.append(rationale_len)
                rationale_percent_0.append(rationale_percent)
                class_distribution[0] += 1
            else:
                text_lens_1.append(text_len)
                rationale_lens_1.append(rationale_len)
                rationale_percent_1.append(rationale_percent)
                class_distribution[1] += 1
                
    all_stats = {"text_lens_0": text_lens_0,
                 "text_lens_1": text_lens_1,
                 "text_lens_all":text_lens_0 + text_lens_1,
                 "rationale_lens_0":rationale_lens_0,
                 "rationale_lens_1":rationale_lens_1,
                 "rationale_lens_all":rationale_lens_0 + rationale_lens_1,
                 "rationale_percent_0": rationale_percent_0,
                 "rationale_percent_1": rationale_percent_1,
                 "rationale_percent_all": rationale_percent_0 + rationale_percent_1,
                 "class_distr":[class_distribution[0]/sum(class_distribution),class_distribution[1]/sum(class_distribution)]
                }
    return all_stats

In [75]:
all_stats = generate_class_stats(train_data_df,test_data_df,val_data_df)
for key,val in all_stats.items():
    print(f"{key}: {avg(val)}")
print(all_stats["class_distr"])

text_lens_0: 3520.54511925252
text_lens_1: 3620.79178040643
text_lens_all: 3582.5492918112745
rationale_lens_0: 108.61937546102779
rationale_lens_1: 110.4877161055505
rationale_lens_all: 109.77497420504643
rationale_percent_0: 0.06850038228444177
rationale_percent_1: 0.06867902337699895
rationale_percent_all: 0.06861087467392883
class_distr: 0.5
[0.381483913328956, 0.618516086671044]


In [77]:
len(train_data_df.iloc[0]['text'].split())

5914